## 💫 MultiTurn Conversation

In [15]:
from typing import TypedDict, Annotated, List
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langgraph.graph import StateGraph, START, END, add_messages
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from IPython.display import Image, display
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver
from langchain_community.tools import TavilySearchResults
from langgraph.prebuilt import ToolNode



In [16]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

class State(TypedDict):
    linkedIn_topic: str
    generated_post: Annotated[List[str], add_messages]
    human_feedback: Annotated[List[str], add_messages]

GENERATED_POST = "generated_post"
HUMAN_FEEDBACK = "human_feedback"

In [23]:
def model(state:State):
    """
    Using the LLM to generate a Linkedin post with human feedback integration.
    """
    print("[model] Generating content")
    linkedIn_topic = state["linkedIn_topic"]
    feedback = state["human_feedback"] if "human_feedback" in state else ["No Feedback yet"]

    prompt = f"""
        LinkedIn Topic: {linkedIn_topic}
        Human Feedback: {feedback[-1] if feedback else "No feedback yet"}

        Generate a structured and well-written LinkedIn post based on the above provided topic.
        Consider previous human feedback to refine the response.

        """

    response = llm.invoke([
        SystemMessage(content="You are an expert LinkedIn content writer"), 
        HumanMessage(content=prompt)
        ])
    
    generated_post = response.content
    print(f"[node_name: mode] Generated post:\n{generated_post}")

    return {GENERATED_POST: [AIMessage(content=generated_post)], HUMAN_FEEDBACK: feedback}

def human_node(state:State):
    """ Human node for review purposes """
    print("\n[node: human_node] awaiting feedback from you...")

    generated_post = state["generated_post"]
    feedback = interrupt({
        "post_generated_by_llm": generated_post,
        "message": "Provide feedback or type 'done' to accept the post"
    })

    print(f"[human_node] Received human feedback: {feedback}")

    if feedback.lower() == "done":
        return Command(goto="end_node", update={HUMAN_FEEDBACK: state["human_feedback"] + ["Finalised"]})

    return Command(goto="model", update={HUMAN_FEEDBACK: state["human_feedback"] + [feedback]})


def end_node(state:State):
    """ Node for printing ouputs """
    print(f"\n[node: end_node] Process finished")
    print(f'\nFinal Generated Post, {state["generated_post"][-1]}')
    print(f'\nFinal Human Feedback, {state["human_feedback"]}')
    return {GENERATED_POST: state["generated_post"], HUMAN_FEEDBACK: state["human_feedback"]}

In [24]:
import uuid


graph = StateGraph(State)
graph.add_node("model", model)
graph.add_node("human_node", human_node)
graph.add_node("end_node", end_node)

graph.add_edge(START, "model")
graph.add_edge("model", "human_node")

graph.set_finish_point("end_node")

memory = MemorySaver()
app = graph.compile(checkpointer=memory)

config = {"configurable":{
    "thread_id" : uuid.uuid4()
}}

In [ ]:
linkedin_topic = input("Enter your topic: ")
initial_state = {
    "linkedIn_topic": linkedin_topic,
    GENERATED_POST: [],
    HUMAN_FEEDBACK: []
}

for chunck in app.stream(initial_state, config):
    print("🔥🔥🔥🔥CHUNCK", chunck)
    for node_id, value in chunck.items():
        # On "interrupt", continuosly ask for feedback from a human
        if (node_id == "__interrupt__"):
            while True:
                feedback = input("Provide feedback or type 'done' to accept the post as is! : ")

                # Resume graph execution with feedback consideration
                app.invoke(Command(resume=feedback), config)

                # feedback == "done", break loop
                if feedback.lower() == "done":
                    break
